### Results data generation

This notebook is the convenient way to run all the algorithms with the customized functionality.
Basically, I've modified the `Base.Evaluator.Evaluator` classes to store the cutoff results.

Ihis notebook run all the results generation at once

In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import os
import traceback
from functools import partial

from Conferences.SIGIR.CMN_our_interface.CMN_RecommenderWrapper import CMN_RecommenderWrapper
from Conferences.SIGIR.CMN_our_interface.test_finetuned_hyperparameters import HYPERPARAMETERS
from Recommender_import_list import UserKNNCFRecommender, ItemKNNCFRecommender
from ParameterTuning.SearchSingleCase import SearchSingleCase
from ParameterTuning.SearchAbstractClass import SearchInputRecommenderArgs
from Base.Evaluation.Evaluator import EvaluatorNegativeItemSample

from Utils.assertions_on_data_for_experiments import assert_implicit_data, assert_disjoint_matrices
from Utils.plot_popularity import plot_popularity_bias, save_popularity_statistics
from Conferences.SIGIR.CMN_our_interface.CiteULike.CiteULikeReader import CiteULikeReader

###########################################

ALGORITHM_NAME = 'CMN'
CONFERENCE_NAME = 'SIGIR'
KNN_SIMILARITY_TO_REPORT_LIST = ['cosine']
TEST_VAL_CUTOFF = 5
TEST_RESULT_CUTOFF = 50
DATASET_LIST = ['citeulike']

USE_EVAL = True
USE_NN = True

###########################################

def evaluate_recommender(
    recommender_class,
    fit_params,
    evaluator_validation,
    evaluator_test,
    URM_train,
    URM_train_last,
    output_file_name_root=None,
    **kwargs,
):
    """
    Helper function to evaluate an instance of `recommender_class` using
    `fit_params` hyperparameters.

    :param recommender_class: Class of the recommender object to optimize, it must be a BaseRecommender type.
    :param fit_params: Dictionary with hyperparameters. Will be passed to `recommender_class.fit()` call.
    :param evaluator_validation: Evaluator object to be used for the validation.
    :param evaluator_test: Evaluator object to be used for the test results.
    :param URM_train: URM matrix that will be used to fit the `recommender_class`.
    :param URM_train_last: URM matrix that will be used for the last fit.
    :param output_file_name_root: Root of the output file name. If none, `recommender_class.RECOMMENDER_NAME` will be used.
    :param kwargs: Other arguments to pass to the `SearchSingleCase().search()` call.
    """
    try:
        if output_file_name_root is None:
            output_file_name_root = recommender_class.RECOMMENDER_NAME

        parameterSearch = SearchSingleCase(
            recommender_class, evaluator_validation, evaluator_test
        )
        recommender_input_args = SearchInputRecommenderArgs([URM_train.copy()])
        recommender_input_args_last_test = SearchInputRecommenderArgs(
            [URM_train_last]
        )
        parameterSearch.search(
            recommender_input_args,
            recommender_input_args_last_test=recommender_input_args_last_test,
            fit_hyperparameters_values=fit_params,
            output_file_name_root=output_file_name_root,
            **kwargs,
        )
    except Exception as e:
        print('On recommender {} Exception {}'.format(recommender_class, str(e)))
        traceback.print_exc()


def run_search_or_eval(
    dataset_name,
    flag_baselines_eval=False,
    flag_DL_article_default=False,
):
    result_folder_path = 'result_experiments/{}/{}_{}/'.format(
        CONFERENCE_NAME, ALGORITHM_NAME, dataset_name
    )
    os.makedirs(result_folder_path, exist_ok=True)

    if dataset_name == 'citeulike':
        dataset = CiteULikeReader(result_folder_path)
    else:
        raise RuntimeError(f'Unsupported dataset: {dataset_name}')

    URM_train = dataset.URM_DICT['URM_train'].copy()
    URM_validation = dataset.URM_DICT['URM_validation'].copy()
    URM_test = dataset.URM_DICT['URM_test'].copy()
    URM_test_negative = dataset.URM_DICT['URM_test_negative'].copy()

    # Ensure IMPLICIT data and DISJOINT sets
    assert_implicit_data(
        [URM_train, URM_validation, URM_test, URM_test_negative]
    )
    if dataset_name == 'citeulike':
        assert_disjoint_matrices([URM_train, URM_validation, URM_test])
        assert_disjoint_matrices([URM_test, URM_test_negative])
    elif dataset_name == 'pinterest':
        assert_disjoint_matrices([URM_train, URM_validation, URM_test])
        assert_disjoint_matrices([URM_train, URM_validation, URM_test_negative])
    else:
        assert_disjoint_matrices(
            [URM_train, URM_validation, URM_test, URM_test_negative]
        )

    algorithm_dataset_string = '{}_{}_'.format(ALGORITHM_NAME, dataset_name)
    plot_popularity_bias(
        [URM_train + URM_validation, URM_test],
        ['Training data', 'Test data'],
        result_folder_path + algorithm_dataset_string + 'popularity_plot',
    )
    save_popularity_statistics(
        [
            URM_train + URM_validation + URM_test,
            URM_train + URM_validation,
            URM_test,
        ],
        ['Full data', 'Training data', 'Test data'],
        result_folder_path + algorithm_dataset_string + 'popularity_statistics'
    )

    evaluator_validation = EvaluatorNegativeItemSample(
        URM_validation, URM_test_negative, cutoff_list=[TEST_VAL_CUTOFF], test_path=result_folder_path
    )
    evaluator_test = EvaluatorNegativeItemSample(
        URM_test, URM_test_negative, cutoff_list=[TEST_RESULT_CUTOFF], test_path=result_folder_path
    )

    metric_to_optimize = 'HIT_RATE'
    if flag_baselines_eval:
        evaluate_recommender_partial = partial(
            evaluate_recommender,
            evaluator_validation=evaluator_validation,
            evaluator_test=evaluator_test,
            URM_train=URM_train,
            URM_train_last=URM_train + URM_validation,
            output_folder_path=result_folder_path,
            resume_from_saved=True,
            save_model='best',
            evaluate_on_test='best',
        )
        for recommender_class in HYPERPARAMETERS:
            fit_params = HYPERPARAMETERS[recommender_class]
            if recommender_class in {
                UserKNNCFRecommender, ItemKNNCFRecommender
            }:
                for similarity_type, params in fit_params[dataset_name].items():
                    evaluate_recommender_partial(
                        recommender_class,
                        params,
                        output_file_name_root=recommender_class.RECOMMENDER_NAME + '_' + similarity_type,
                    )
            else:
                evaluate_recommender_partial(
                    recommender_class, fit_params[dataset_name]
                )

    if flag_DL_article_default:
        try:
            CMN_article_hyperparameters = {
                'epochs_gmf': 100,
                'hops': 3,
                'neg_samples': 4,
                'reg_l2_cmn': 1e-1,
                'reg_l2_gmf': 1e-4,
                'pretrain': True,
                'learning_rate': 1e-3,
                'verbose': False,
            }
            if dataset_name == 'citeulike':
                CMN_article_hyperparameters['epochs'] = 50
                CMN_article_hyperparameters['batch_size'] = 128
                CMN_article_hyperparameters['embed_size'] = 50
            elif dataset_name == 'epinions':
                CMN_article_hyperparameters['epochs'] = 45
                CMN_article_hyperparameters['batch_size'] = 128
                CMN_article_hyperparameters['embed_size'] = 40
            else:
                CMN_article_hyperparameters['epochs'] = 5
                CMN_article_hyperparameters['batch_size'] = 256
                CMN_article_hyperparameters['embed_size'] = 50

            CMN_earlystopping_hyperparameters = {
                'validation_every_n': 5,
                'stop_on_validation': True,
                'evaluator_object': evaluator_validation,
                'lower_validations_allowed': 5,
                'validation_metric': metric_to_optimize
            }

            parameterSearch = SearchSingleCase(
                CMN_RecommenderWrapper,
                evaluator_validation=evaluator_validation,
                evaluator_test=evaluator_test
            )
            recommender_input_args = SearchInputRecommenderArgs(
                CONSTRUCTOR_POSITIONAL_ARGS = [URM_train],
                FIT_KEYWORD_ARGS = CMN_earlystopping_hyperparameters
            )
            recommender_input_args_last_test = recommender_input_args.copy()
            recommender_input_args_last_test.CONSTRUCTOR_POSITIONAL_ARGS[0] = URM_train + URM_validation

            parameterSearch.search(
                recommender_input_args,
                recommender_input_args_last_test=recommender_input_args_last_test,
                fit_hyperparameters_values=CMN_article_hyperparameters,
                output_folder_path=result_folder_path,
                resume_from_saved=True,
                output_file_name_root=CMN_RecommenderWrapper.RECOMMENDER_NAME
            )
        except Exception as e:
            print(
                'On recommender {} Exception {}'.format(
                    CMN_RecommenderWrapper, str(e)
                )
            )
            traceback.print_exc()

if __name__ == '__main__':
    for dataset_name in DATASET_LIST:
        run_search_or_eval(
            dataset_name,
            flag_baselines_eval=USE_EVAL,
            flag_DL_article_default=USE_NN
        )

CiteULikeReader: Attempting to load pre-splitted data
CiteULikeReader: Dataset loaded
Assertion assert_implicit_data: Passed
Assertion assert_disjoint_matrices: Passed
Assertion assert_disjoint_matrices: Passed
SearchSingleCase: Resuming 'TopPopRecommender'... Loaded 1 configurations.
TopPopRecommender: URM Detected 8 (0.05 %) cold items.
SearchSingleCase: Evaluation with constructor data for final test. Using best config: {}
EvaluatorNegativeItemSample: Processed 5551 (100.0%) in 10.10 sec. Users per second: 549
SearchSingleCase: Best config evaluated with evaluator_test with constructor data for final test. Config: {} - results:
CUTOFF: 50 - PRECISION: 0.0129202, PRECISION_RECALL_MIN_DEN: 0.6460097, RECALL: 0.6460097, MAP: 0.0026518, MAP_MIN_DEN: 0.1325899, MRR: 0.1325899, NDCG: 0.2334525, F1: 0.0253337, HIT_RATE: 0.6460097, ARHR_ALL_HITS: 0.1325899, NOVELTY: 0.0403519, AVERAGE_POPULARITY: 0.0563261, DIVERSITY_MEAN_INTER_LIST: 0.9944240, DIVERSITY_HERFINDAHL: 0.9998849, COVERAGE_ITEM

In [2]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import os
import traceback
from functools import partial
from collections import namedtuple

from Recommender_import_list import UserKNNCFRecommender, ItemKNNCFRecommender
from Base.Evaluation.Evaluator import EvaluatorHoldout
from Conferences.RecSys.SpectralCF_our_interface.SpectralCF_RecommenderWrapper import SpectralCF_RecommenderWrapper
from Conferences.RecSys.SpectralCF_our_interface.test_finetuned_hyperparameters import HYPERPARAMETERS
from ParameterTuning.SearchSingleCase import SearchSingleCase
from ParameterTuning.SearchAbstractClass import SearchInputRecommenderArgs
from Utils.assertions_on_data_for_experiments import assert_implicit_data, assert_disjoint_matrices
from Utils.plot_popularity import plot_popularity_bias, save_popularity_statistics
from Conferences.RecSys.SpectralCF_our_interface.Movielens1M.Movielens1MReader import Movielens1MReader
from Conferences.RecSys.SpectralCF_our_interface.AmazonInstantVideo.AmazonInstantVideoReader import AmazonInstantVideoReader
from skopt.space import Real, Integer, Categorical
from ParameterTuning.SearchBayesianSkopt import SearchBayesianSkopt
from ParameterTuning.SearchSingleCase import SearchSingleCase
from ParameterTuning.SearchAbstractClass import SearchInputRecommenderArgs

###################################################################################

# Disable GPU if the script fails due to OOM error
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

ALGORITHM_NAME = 'SpectralCF'
CONFERENCE_NAME = 'RecSys'
KNN_SIMILARITY_TO_REPORT_LIST = ['cosine']
DATASET_LIST = ['movielens1m_ours', 'amazon_instant_video']
TEST_VAL_CUTOFF = 5
TEST_RESULT_CUTOFF = 50

USE_EVAL = True
USE_NN = False
USE_NN_TUNED = True

CustomRecommenderName = namedtuple('CustomRecommenderName', ['RECOMMENDER_NAME'])

###################################################################################


def evaluate_recommender(
    recommender_class,
    fit_params,
    evaluator_validation,
    evaluator_test,
    URM_train,
    URM_train_last,
    output_file_name_root=None,
    **kwargs,
):
    """
    Helper function to evaluate an instance of `recommender_class` using
    `fit_params` hyperparameters.

    :param recommender_class: Class of the recommender object to optimize, it must be a BaseRecommender type.
    :param fit_params: Dictionary with hyperparameters. Will be passed to `recommender_class.fit()` call.
    :param evaluator_validation: Evaluator object to be used for the validation.
    :param evaluator_test: Evaluator object to be used for the test results.
    :param URM_train: URM matrix that will be used to fit the `recommender_class`.
    :param URM_train_last: URM matrix that will be used for the last fit.
    :param output_file_name_root: Root of the output file name. If none, `recommender_class.RECOMMENDER_NAME` will be used.
    :param kwargs: Other arguments to pass to the `SearchSingleCase().search()` call.
    """
    try:
        if output_file_name_root is None:
            output_file_name_root = recommender_class.RECOMMENDER_NAME

        parameterSearch = SearchSingleCase(
            recommender_class, evaluator_validation, evaluator_test
        )
        recommender_input_args = SearchInputRecommenderArgs([URM_train.copy()])
        recommender_input_args_last_test = SearchInputRecommenderArgs(
            [URM_train_last]
        )
        parameterSearch.search(
            recommender_input_args,
            recommender_input_args_last_test=recommender_input_args_last_test,
            fit_hyperparameters_values=fit_params,
            output_file_name_root=output_file_name_root,
            **kwargs,
        )
    except Exception as e:
        print(
            'On recommender {} Exception {}'.format(recommender_class, str(e))
        )
        traceback.print_exc()


def runParameterSearch_SpectralCF(
    recommender_class,
    URM_train,
    earlystopping_hyperparameters,
    output_file_name_root,
    URM_train_last_test=None,
    n_cases=35,
    n_random_starts=5,
    evaluator_validation=None,
    evaluator_test=None,
    metric_to_optimize='RECALL',
    output_folder_path='result_experiments/',
):
    # If directory does not exist, create
    if not os.path.exists(output_folder_path):
        os.makedirs(output_folder_path)

    parameterSearch = SearchBayesianSkopt(
        recommender_class,
        evaluator_validation=evaluator_validation,
        evaluator_test=evaluator_test,
    )

    if recommender_class is SpectralCF_RecommenderWrapper:
        hyperparameters_range_dictionary = {}
        hyperparameters_range_dictionary['batch_size'] = Categorical(
            [128, 256, 512, 1024, 2048]
        )
        hyperparameters_range_dictionary['embedding_size'] =  Categorical(
            [4, 8, 16, 32]
        )
        hyperparameters_range_dictionary['decay'] = Real(
            low=1e-5, high=1e-1, prior='log-uniform'
        )
        hyperparameters_range_dictionary['learning_rate'] = Real(
            low=1e-5, high=1e-2, prior='log-uniform'
        )
        hyperparameters_range_dictionary['k'] = Integer(low=1, high=6)

        recommender_input_args = SearchInputRecommenderArgs(
            CONSTRUCTOR_POSITIONAL_ARGS=[URM_train],
            CONSTRUCTOR_KEYWORD_ARGS={},
            FIT_POSITIONAL_ARGS=[],
            FIT_KEYWORD_ARGS=earlystopping_hyperparameters,
        )

    if URM_train_last_test is not None:
        recommender_input_args_last_test = recommender_input_args.copy()
        recommender_input_args_last_test.CONSTRUCTOR_POSITIONAL_ARGS[0] = URM_train_last_test
    else:
        recommender_input_args_last_test = None

    parameterSearch.search(
        recommender_input_args,
        parameter_search_space=hyperparameters_range_dictionary,
        n_cases=n_cases,
        n_random_starts=n_random_starts,
        resume_from_saved=True,
        output_folder_path=output_folder_path,
        output_file_name_root=output_file_name_root,
        metric_to_optimize=metric_to_optimize,
        recommender_input_args_last_test=recommender_input_args_last_test,
    )


def run_search_or_eval(
    dataset_name,
    flag_baselines_eval=False,
    flag_DL_article_default = False,
    flag_DL_tuned = False,
):

    result_folder_path = 'result_experiments/{}/{}_{}/'.format(
        CONFERENCE_NAME, ALGORITHM_NAME, dataset_name)

    os.makedirs(result_folder_path, exist_ok=True)

    if dataset_name == 'movielens1m_ours':
        dataset = Movielens1MReader(
            result_folder_path,
            type ='ours'
        )
    elif dataset_name == 'amazon_instant_video':
        dataset = AmazonInstantVideoReader(result_folder_path)
    else:
        raise RuntimeError(f'Unsupported dataset: {dataset_name}')

    URM_train = dataset.URM_DICT['URM_train'].copy()
    URM_validation = dataset.URM_DICT['URM_validation'].copy()
    URM_test = dataset.URM_DICT['URM_test'].copy()

    # Ensure IMPLICIT data and DISJOINT sets
    assert_implicit_data([URM_train, URM_validation, URM_test])
    assert_disjoint_matrices([URM_train, URM_validation, URM_test])

    algorithm_dataset_string = '{}_{}_'.format(ALGORITHM_NAME, dataset_name)
    plot_popularity_bias(
        [URM_train + URM_validation, URM_test],
        ['Training data', 'Test data'],
        result_folder_path + algorithm_dataset_string + 'popularity_plot'
    )
    save_popularity_statistics(
        [
            URM_train + URM_validation + URM_test,
            URM_train + URM_validation,
            URM_test
        ],
        ['Full data', 'Training data', 'Test data'],
        result_folder_path + algorithm_dataset_string + 'popularity_statistics'
    )
    cutoff_list_validation = [TEST_VAL_CUTOFF]
    cutoff_list_test = [TEST_RESULT_CUTOFF]

    evaluator_validation = EvaluatorHoldout(
        URM_validation, cutoff_list=cutoff_list_validation, test_path=result_folder_path
    )
    evaluator_test = EvaluatorHoldout(
        URM_test, cutoff_list=cutoff_list_test, test_path=result_folder_path
    )

    metric_to_optimize = 'RECALL'
    n_cases = 50
    n_random_starts = 15
    if flag_baselines_eval:
        evaluate_recommender_partial = partial(
            evaluate_recommender,
            evaluator_validation=evaluator_validation,
            evaluator_test=evaluator_test,
            URM_train=URM_train,
            URM_train_last=URM_train + URM_validation,
            output_folder_path=result_folder_path,
            resume_from_saved=True,
            save_model='best',
            evaluate_on_test='best',
        )

        for recommender_class in HYPERPARAMETERS:
            fit_params = HYPERPARAMETERS[recommender_class]
            if recommender_class in {
                UserKNNCFRecommender, ItemKNNCFRecommender
            }:
                for similarity_type, params in fit_params[dataset_name].items():
                    evaluate_recommender_partial(
                        recommender_class,
                        params,
                        output_file_name_root=recommender_class.RECOMMENDER_NAME + '_' + similarity_type,
                    )
            else:
                evaluate_recommender_partial(
                    recommender_class, fit_params[dataset_name]
                )

    if flag_DL_article_default:
        try:
            spectralCF_article_hyperparameters = {
                'batch_size': 1024,
                'embedding_size': 16,
                'decay': 0.001,
                'k': 3,
                'learning_rate': 1e-3,
            }

            if dataset_name == 'movielens1m_ours':
                spectralCF_article_hyperparameters['epochs'] = 600
            else:
                spectralCF_article_hyperparameters['epochs'] = 425

            spectralCF_earlystopping_hyperparameters = {
                'validation_every_n': 5,
                'stop_on_validation': True,
                'lower_validations_allowed': 5,
                'evaluator_object': evaluator_validation,
                'validation_metric': metric_to_optimize,
                'epochs_min': 400,
            }

            parameterSearch = SearchSingleCase(
                SpectralCF_RecommenderWrapper,
                evaluator_validation=evaluator_validation,
                evaluator_test=evaluator_test,
            )
            recommender_input_args = SearchInputRecommenderArgs(
                CONSTRUCTOR_POSITIONAL_ARGS=[URM_train],
                FIT_KEYWORD_ARGS=spectralCF_earlystopping_hyperparameters,
            )

            recommender_input_args_last_test = recommender_input_args.copy()
            recommender_input_args_last_test.CONSTRUCTOR_POSITIONAL_ARGS[0] = URM_train + URM_validation

            parameterSearch.search(
                recommender_input_args,
                recommender_input_args_last_test=recommender_input_args_last_test,
                fit_hyperparameters_values=spectralCF_article_hyperparameters,
                output_folder_path=result_folder_path,
                resume_from_saved=True,
                output_file_name_root=SpectralCF_RecommenderWrapper.RECOMMENDER_NAME + '_article_default',
            )
        except Exception as e:
            print(
                'On recommender {} Exception {}'.format(
                    SpectralCF_RecommenderWrapper, str(e)
                )
            )
            traceback.print_exc()

    if flag_DL_tuned:
        try:
            if dataset_name == 'movielens1m_original':
                spectralCF_article_hyperparameters = {
                    'batch_size': 2048,
                    'embedding_size': 16,
                    'decay': 3.2e-3,
                    'learning_rate': 7e-3,
                    'k': 3,
                    'epochs': 350,
                }
            elif dataset_name == 'movielens1m_ours':
                spectralCF_article_hyperparameters = {
                    'batch_size': 2048,
                    'embedding_size': 4,
                    'decay': 3.06e-2,
                    'learning_rate': 8.83e-4,
                    'k': 2,
                    'epochs': 805,
                }
            elif dataset_name == 'hetrec':
                spectralCF_article_hyperparameters = {
                    'batch_size': 512,
                    'embedding_size': 16,
                    'decay': 1.8e-3,
                    'learning_rate': 5.35e-3,
                    'k': 2,
                    'epochs': 265,
                }
            else:
                spectralCF_article_hyperparameters = {
                    'batch_size': 1024,
                    'embedding_size': 8,
                    'decay': 2.78e-4,
                    'learning_rate': 9.68e-3,
                    'k': 3,
                    'epochs': 445,
                }

            spectralCF_earlystopping_hyperparameters = {
                'validation_every_n': 5,
                'stop_on_validation': True,
                'lower_validations_allowed': 5,
                'evaluator_object': evaluator_validation,
                'validation_metric': metric_to_optimize,
                'epochs_min': 400,
            }

            parameterSearch = SearchSingleCase(
                SpectralCF_RecommenderWrapper,
                evaluator_validation=evaluator_validation,
                evaluator_test=evaluator_test,
            )
            recommender_input_args = SearchInputRecommenderArgs(
                CONSTRUCTOR_POSITIONAL_ARGS=[URM_train],
                FIT_KEYWORD_ARGS=spectralCF_earlystopping_hyperparameters,
            )

            recommender_input_args_last_test = recommender_input_args.copy()
            recommender_input_args_last_test.CONSTRUCTOR_POSITIONAL_ARGS[0] = URM_train + URM_validation

            parameterSearch.search(
                recommender_input_args,
                recommender_input_args_last_test=recommender_input_args_last_test,
                fit_hyperparameters_values=spectralCF_article_hyperparameters,
                output_folder_path=result_folder_path,
                resume_from_saved=True,
                output_file_name_root=SpectralCF_RecommenderWrapper.RECOMMENDER_NAME,
            )
        except Exception as e:
            print(
                'On recommender {} Exception {}'.format(
                    SpectralCF_RecommenderWrapper, str(e)
                )
            )
            traceback.print_exc()


if __name__ == '__main__':

    for dataset_name in DATASET_LIST:
        run_search_or_eval(
            dataset_name,
            flag_baselines_eval=USE_EVAL,
            flag_DL_article_default=USE_NN,
            flag_DL_tuned=USE_NN_TUNED,
        )

Dataset_Movielens1M: Attempting to load pre-splitted data
Dataset_Movielens1M: Dataset loaded
DATASET   -> users: 6039 	items: 3882 	ratings:   225473 	density: 0.962%
URM_train -> users: 6039 	items: 3882 	ratings:   161797 	density: 0.690%
URM_test  -> users: 6039 	items: 3882 	ratings:    45200 	density: 0.193%
URM_validation-> users: 6039 	items: 3882 	ratings:    18476 	density: 0.079%
Assertion assert_implicit_data: Passed
Assertion assert_disjoint_matrices: Passed
EvaluatorHoldout: Ignoring 5881 ( 2.6%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 5960 ( 1.3%) Users that have less than 1 test interactions
SearchSingleCase: Resuming 'TopPopRecommender'... Loaded 1 configurations.
TopPopRecommender: URM Detected 26 (0.43 %) cold users.
TopPopRecommender: URM Detected 725 (18.68 %) cold items.
SearchSingleCase: Resuming 'TopPopRecommender'... Result on last already available.
SearchSingleCase: Resuming 'UserKNNCFRecommender_cosine'... Loaded 1 configurat